In [0]:
!pip install allennlp

     |████████████████████████████████| 5.6MB 3.3MB/s 
     |████████████████████████████████| 583kB 33.2MB/s 
     |████████████████████████████████| 133kB 56.3MB/s 
     |████████████████████████████████| 245kB 63.1MB/s 
     |████████████████████████████████| 51kB 29.7MB/s 
     |████████████████████████████████| 133kB 47.9MB/s 
     |████████████████████████████████| 51kB 24.4MB/s 
     |████████████████████████████████| 245kB 52.9MB/s 
     |████████████████████████████████| 1.6MB 44.6MB/s 
     |████████████████████████████████| 143kB 50.6MB/s 
     |████████████████████████████████| 2.2MB 40.6MB/s 
     |████████████████████████████████| 2.3MB 40.8MB/s 
     |████████████████████████████████| 92kB 33.4MB/s 
     |████████████████████████████████| 51kB 27.8MB/s 
     |████████████████████████████████| 204kB 50.0MB/s 
     |████████████████████████████████| 61kB 28.1MB/s 
     |████████████████████████████████| 102kB 35.7MB/s 
     |████████████████████████████████| 102kB 32.5MB/s

In [0]:
from nltk.tokenize import word_tokenize
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, LabelField
from typing import Iterator, List, Dict
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper, Seq2VecEncoder
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.data.vocabulary import Vocabulary
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask
from allennlp.modules.token_embedders import Embedding
from allennlp.data.iterators import BasicIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.predictors.predictor import Predictor

import torch.optim as optim
import numpy as np
import nltk

import pickle
import torch

nltk.download('punkt')

class VerbDatasetReader(DatasetReader):

    def __init__(self,sentence_indexers:Dict[str,TokenIndexer] = None)-> None:
        super().__init__(lazy=False)
        self.sentence_indexers=sentence_indexers or {"sentence":SingleIdTokenIndexer()}

    def text_to_instance(self, sentence:List[List],labels:str = None)->Instance:
        sent_tokenized=[]
        if labels=="NOT ENOUGH INFO":
            for word in sentence:
                sent_tokenized.append(Token(word))
            sentence_field=TextField(sent_tokenized,self.sentence_indexers)
            fields = {'sentence': sentence_field, 'labels': LabelField(labels)}
            return Instance(fields)
        for sent in sentence:
            for word in word_tokenize(sent):
                sent_tokenized.append(Token(word))
        sentence_field=TextField(sent_tokenized,self.sentence_indexers)
        fields={'sentence':sentence_field,'labels':LabelField(labels)}
        return Instance(fields)

    def _read(self, file_path: str)->Iterator[Instance]:
        mlinput_merge=pickle.load(open(file_path,'rb'))
        for entry in mlinput_merge[:64]:
            sentence_input=[entry['claim']]
            #if entry['label']=='REFUTES'
            if entry['label']=="NOT ENOUGH INFO":
                #yield self.text_to_instance(sentence_input,entry['label'])
                continue
            else:
                i=0
                for sent in entry['evidence']:
                    if i<=5:
                        full_sent=' '.join(sent)
                        sentence_input.append(full_sent)
                        i+=1
                    else:
                        break
                yield self.text_to_instance(sentence_input,entry['label'])

class Lstm(Model):
    def __init__(self,
                 word_embeddings:TextFieldEmbedder,
                 encoder:Seq2VecEncoder,
                 vocab:Vocabulary)->None:
        super().__init__(vocab)
        self.word_embeddings=word_embeddings
        self.encoder=encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()

    def forward(self,
                sentence:Dict[str,torch.Tensor],
                labels:torch.Tensor==None)->Dict[str,torch.Tensor]:
        mask=get_text_field_mask(sentence)
        embeddings = self.word_embeddings(sentence)
        encoder_out = self.encoder(embeddings, mask)
        tag_logits=self.hidden2tag(encoder_out)
        output = {"tag_logits": tag_logits}
        if labels is not None:
            self.accuracy(tag_logits, labels)
            print(labels.shape)
            loss=torch.nn.CrossEntropyLoss()
            output['loss'] = loss(tag_logits,labels)
        return output

    def get_metrics(self, reset: bool = False) -> Dict[str, float]:
        return {"accuracy": self.accuracy.get_metric(reset)}

reader=VerbDatasetReader()


print("start reading data")
train_dataset=reader.read('mlinput_merge_with_entity.txt')
print("data reading finished")

vocab=Vocabulary.from_instances(train_dataset)

EMBEDDING_DIM = 300
HIDDEN_DIM = 6

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"sentence": token_embedding})
lstm = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model = Lstm(word_embeddings, lstm, vocab)

optimizer = optim.SGD(model.parameters(), lr=0.1)

iterator=BasicIterator()

iterator.index_with(vocab)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=train_dataset[:64],
                  patience=3,
                  num_epochs=10)
print("start training model")
trainer.train()
print("training finished")

print("start saving the weights")
with open("test.th", 'wb') as f:
    torch.save(model.state_dict(), f)
print("saving finished")
print("start saving vocab")
vocab.save_to_files("vocabulary")
print("saving finished")


#predictor = SentenceTaggerPredictor(model, dataset_reader=reader)
predictor=Predictor(model,dataset_reader=reader)

print('start doing prediction')
# for i in range(0,len(train_dataset),4):
#     print(train_dataset[i])
#     tag_logits = predictor.predict_instance(train_dataset[i])
#     softmax=torch.nn.Softmax()
#     print(tag_logits)
#     tag_ids = softmax(torch.FloatTensor(tag_logits['tag_logits']))
#     print(tag_ids)
    # tag_ids = np.argmax(tag_ids, axis=1)
    # print(vars(tag_ids))
#print(tag_ids)
tag_logits=predictor.predict_batch_instance(train_dataset[:32])
for i in range(32):
    print(tag_logits[i])
    print(train_dataset[i])

print("predicting finished")



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.




0it [00:00, ?it/s]

start reading data




1it [00:01,  1.83s/it]
47it [00:01, 23.51it/s]
100%|██████████| 47/47 [00:00<00:00, 3785.33it/s]unable to check gpu_memory_mb(), continuing
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/allennlp/common/util.py", line 379, in gpu_memory_mb
    encoding='utf-8')
  File "/usr/lib/python3.6/subprocess.py", line 336, in check_output
    **kwargs).stdout
  File "/usr/lib/python3.6/subprocess.py", line 418, in run
    output=stdout, stderr=stderr)
subprocess.CalledProcessError: Command '['nvidia-smi', '--query-gpu=memory.used', '--format=csv,nounits,noheader']' returned non-zero exit status 9.

  0%|          | 0/2 [00:00<?, ?it/s]

data reading finished
start training model
torch.Size([32])


...

torch.Size([32])
torch.Size([15])
torch.Size([32])



accuracy: 0.6875, loss: 0.6171 ||:  50%|█████     | 1/2 [00:00<00:00,  4.24it/s]
accuracy: 0.7234, loss: 0.5765 ||: 100%|██████████| 2/2 [00:00<00:00,  4.32it/s]


torch.Size([15])



  0%|          | 0/2 [00:00<?, ?it/s]
accuracy: 0.7234, loss: 0.6019 ||: 100%|██████████| 2/2 [00:00<00:00, 37.76it/s]Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.


torch.Size([32])
torch.Size([15])
training finished
start saving the weights
saving finished
start saving vocab
saving finished
start doing prediction
torch.Size([32])
{'tag_logits': [0.4784800410270691, -0.34949320554733276]}
Instance with fields:
 	 sentence: TextField of length 132 with text: 
 		[Selena, recorded, music, ., She, also, ranks, among, the, most, influential, Latin, artists, of,
		all-time, and, is, credited, for, catapulting, a, music, genre, into, the, mainstream, market, .,
		In, 1994, ,, Selena, released, Amor, Prohibido, ,, which, became, one, of, the, best-selling, Latin,
		albums, in, the, United, States, ., One, of, its, singles, ,, ``, Como, La, Flor, '', ,, became,
		one, of, her, most, popular, signature, songs, ., However, ,, her, popularity, grew, after, she,
		won, the, Tejano, Music, Award, for, Female, Vocalist, of, the, Year, in, 1987, ,, which, she, won,
		nine, consecutive, times, ., Selena, signed, with, EMI, Latin, in, 1989, and, released, her, sel

In [0]:
from nltk.tokenize import word_tokenize
from allennlp.data.tokenizers import Token
from allennlp.data.fields import TextField, LabelField
from typing import Iterator, List, Dict
from allennlp.data import Instance
from allennlp.data.dataset_readers import DatasetReader
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.models import Model
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper, Seq2VecEncoder
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.data.vocabulary import Vocabulary
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.nn.util import get_text_field_mask
from allennlp.modules.token_embedders import Embedding
from allennlp.data.iterators import BasicIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.predictors.predictor import Predictor

import torch.optim as optim
import numpy as np

import pickle
import torch

class ProjDataSetReader(DatasetReader):

    def __init__(self, sentence_indexers: Dict[str, TokenIndexer] = None) -> None:
        super().__init__(lazy=False)
        self.sentence_indexers = sentence_indexers or {"sentence": SingleIdTokenIndexer()}

    def text_to_instance(self, claim:str,evidence:List[str],label:str=None)->Instance:
        tobeField=[]
        for claim_word in claim.split(' '):
            tobeField.append(Token(claim_word))
        claim_field=TextField(tobeField,self.sentence_indexers)
        tobeField=[]
        for evidence_sent in evidence:
            for evidence_word in evidence_sent.split(' '):
                tobeField.append(Token(evidence_word))
        evidence_field=TextField(tobeField,self.sentence_indexers)
        fields={'claim':claim_field,'evidence':evidence_field,'label':LabelField(label)}
        return Instance(fields)

    def _read(self, file_path: str)->Iterator[Instance]:
        train_input=pickle.load(open(file_path,'rb'))
        pre = ""
        for value in train_input:
            claim = value['claim']
            label = value['label']
            if label == "NOT ENOUGH INFO":
                #                     continue
                #instance = claim + ' ' + pre
                evidence=[pre]
                yield self.text_to_instance(claim,evidence,label)
            else:
                i = 0
                full_evidence=[]
                for e in value['evidence']:
                    pre = ' '.join(e)
                    #instance += ' ' + pre
                    full_evidence.append(pre)
                    i += 1
                    if i >= 3:
                        break
                yield self.text_to_instance(claim,full_evidence,label)

class Lstm(Model):
    def __init__(self,
                 word_embedding:TextFieldEmbedder,
                 encoder:Seq2VecEncoder,
                 vocab:Vocabulary)->None:
        super().__init__(vocab)
        self.word_embedding=word_embedding
        self.encoder=encoder
        self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),out_features=vocab.get_vocab_size('labels'))
        self.accuracy = CategoricalAccuracy()

    def forward(self,
                claim:Dict[str,torch.Tensor],
                evidence:Dict[str,torch.Tensor],
                label:torch.Tensor==None)->Dict[str,torch.Tensor]:
        claim_mask=get_text_field_mask(claim)
        claim_embedding=self.word_embedding(claim)
        claim_encoder_out=self.encoder(claim_embedding,claim_mask)
        claim_tag_logits=self.hidden2tag(claim_encoder_out)
        evidence_mask=get_text_field_mask(evidence)
        evidence_embedding=self.word_embedding(evidence)
        evidence_encoder_out=self.encoder(evidence_embedding,evidence_mask)
        evidence_tag_logits=self.hidden2tag(evidence_encoder_out)
        tag_logits=claim_tag_logits+evidence_tag_logits
        output={'tag_logits':tag_logits}
        if label is not None:
            self.accuracy(tag_logits,label)
            loss=torch.nn.CrossEntropyLoss()
            output['loss']=loss(tag_logits,label)
        return output

    def get_metrics(self, reset: bool = False)->Dict[str,float]:
        return {'accuracy':self.accuracy.get_metric(reset)}

reader=ProjDataSetReader()

print("start reading data")
train_dataset=reader.read('mlinput_merge.txt')
print("data reading finished")

vocab=Vocabulary.from_instances(train_dataset)

EMBEDDING_DIM = 300
HIDDEN_DIM = 6

token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"sentence": token_embedding})
lstm = PytorchSeq2VecWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True))

model = Lstm(word_embeddings, lstm, vocab)

if torch.cuda.is_available():
  cuda_device=0
  model=model.cuda(cuda_device)
else:
  cuda_device=-1
 



0it [00:00, ?it/s]

start reading data



1it [00:00,  2.67it/s]
1343it [00:00,  3.81it/s]
2670it [00:00,  5.45it/s]
3991it [00:00,  7.78it/s]
5372it [00:00, 11.11it/s]
6753it [00:00, 15.87it/s]
8059it [00:00, 22.66it/s]
9285it [00:04, 31.44it/s]
10498it [00:04, 44.86it/s]
11775it [00:04, 64.00it/s]
13092it [00:05, 91.23it/s]
14408it [00:05, 129.95it/s]
15759it [00:05, 184.87it/s]
17057it [00:05, 262.50it/s]
18372it [00:05, 371.82it/s]
19698it [00:05, 524.86it/s]
21024it [00:05, 737.28it/s]
22357it [00:05, 1028.86it/s]
23698it [00:05, 1423.00it/s]
25017it [00:05, 1941.10it/s]
26330it [00:06, 2607.75it/s]
27641it [00:06, 3426.87it/s]
28972it [00:06, 4408.82it/s]
30283it [00:06, 5460.25it/s]
31570it [00:06, 6589.96it/s]
32893it [00:06, 7757.66it/s]
34191it [00:06, 8821.39it/s]
35487it [00:06, 9731.73it/s]
36779it [00:06, 10491.45it/s]
38068it [00:06, 10875.75it/s]
39381it [00:07, 11465.11it/s]
40740it [00:07, 12027.03it/s]
42049it [00:07, 12326.47it/s]
43381it [00:07, 12607.31it/s]
44726it [00:07, 12847.51it/s]
46070it [00:07, 

data reading finished



  0%|          | 0/4546 [00:00<?, ?it/s]

start training model



accuracy: 0.2500, loss: 1.1623 ||:   0%|          | 1/4546 [00:00<20:19,  3.73it/s]
accuracy: 0.4271, loss: 1.0893 ||:   0%|          | 6/4546 [00:00<14:42,  5.14it/s]
accuracy: 0.4844, loss: 1.0512 ||:   0%|          | 10/4546 [00:00<10:53,  6.94it/s]
accuracy: 0.5208, loss: 1.0213 ||:   0%|          | 15/4546 [00:00<08:06,  9.31it/s]
accuracy: 0.5328, loss: 1.0127 ||:   0%|          | 20/4546 [00:00<06:12, 12.14it/s]
accuracy: 0.5188, loss: 1.0232 ||:   1%|          | 25/4546 [00:00<04:53, 15.41it/s]
accuracy: 0.5250, loss: 1.0192 ||:   1%|          | 30/4546 [00:00<03:56, 19.13it/s]
accuracy: 0.5286, loss: 1.0170 ||:   1%|          | 35/4546 [00:01<03:15, 23.04it/s]
accuracy: 0.5211, loss: 1.0229 ||:   1%|          | 40/4546 [00:01<02:46, 27.13it/s]
accuracy: 0.5243, loss: 1.0213 ||:   1%|          | 45/4546 [00:01<02:27, 30.52it/s]
accuracy: 0.5281, loss: 1.0181 ||:   1%|          | 50/4546 [00:01<02:13, 33.66it/s]
accuracy: 0.5290, loss: 1.0171 ||:   1%|          | 55/4546 [00:01

KeyboardInterrupt: ignored

In [0]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

iterator=BasicIterator()

iterator.index_with(vocab)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=train_dataset[:64],
                  patience=3,
                  num_epochs=10,
                  cuda_device=cuda_device)
print("start training model")
trainer.train()
print("training finished")

batch_generator = iterator(train_dataset, num_epochs=1, shuffle=False)
model.eval()
with torch.no_grad():
    for batch in batch_generator:
      logits = model(**batch)
      print(logits)



  0%|          | 0/4546 [00:00<?, ?it/s]

start training model




accuracy: 0.6649, loss: 0.6962 ||:   0%|          | 1/4546 [00:00<19:15,  3.93it/s]

accuracy: 0.6649, loss: 0.6475 ||:   0%|          | 6/4546 [00:00<13:57,  5.42it/s]

accuracy: 0.6649, loss: 0.6657 ||:   0%|          | 11/4546 [00:00<10:16,  7.36it/s]

accuracy: 0.6651, loss: 0.6431 ||:   0%|          | 16/4546 [00:00<07:41,  9.82it/s]

accuracy: 0.6651, loss: 0.6595 ||:   0%|          | 22/4546 [00:00<05:48, 12.99it/s]

accuracy: 0.6652, loss: 0.6545 ||:   1%|          | 28/4546 [00:00<04:28, 16.81it/s]

accuracy: 0.6652, loss: 0.6597 ||:   1%|          | 34/4546 [00:00<03:32, 21.19it/s]

accuracy: 0.6653, loss: 0.6650 ||:   1%|          | 40/4546 [00:01<02:53, 26.02it/s]

accuracy: 0.6653, loss: 0.6661 ||:   1%|          | 46/4546 [00:01<02:26, 30.65it/s]

accuracy: 0.6654, loss: 0.6709 ||:   1%|          | 51/4546 [00:01<02:11, 34.29it/s]

accuracy: 0.6655, loss: 0.6725 ||:   1%|▏         | 57/4546 [00:01<01:57, 38.22it/s]

accuracy: 0.6656, loss: 0.6671 ||:   1%|▏         | 63

KeyboardInterrupt: ignored

In [0]:

optimizer = optim.SGD(model.parameters(), lr=0.00001)

iterator=BasicIterator()

iterator.index_with(vocab)

trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=train_dataset[:64],
                  patience=3,
                  num_epochs=3,
                  cuda_device=cuda_device)
print("start training model")
trainer.train()
print("training finished")

batch_generator = iterator(train_dataset, num_epochs=1, shuffle=False)
model.eval()
with torch.no_grad():
    for batch in batch_generator:
      logits = model(**batch)
      print(logits)




  0%|          | 0/4546 [00:00<?, ?it/s]

start training model





accuracy: 0.6991, loss: 0.6454 ||:   0%|          | 1/4546 [00:00<20:06,  3.77it/s]


accuracy: 0.6990, loss: 0.7023 ||:   0%|          | 6/4546 [00:00<14:33,  5.20it/s]


accuracy: 0.6990, loss: 0.7069 ||:   0%|          | 11/4546 [00:00<10:39,  7.09it/s]


accuracy: 0.6990, loss: 0.7001 ||:   0%|          | 16/4546 [00:00<07:55,  9.53it/s]


accuracy: 0.6990, loss: 0.6987 ||:   0%|          | 22/4546 [00:00<05:58, 12.61it/s]


accuracy: 0.6988, loss: 0.7107 ||:   1%|          | 28/4546 [00:00<04:37, 16.29it/s]


accuracy: 0.6989, loss: 0.7003 ||:   1%|          | 33/4546 [00:00<03:41, 20.35it/s]


accuracy: 0.6990, loss: 0.6869 ||:   1%|          | 39/4546 [00:01<03:01, 24.85it/s]


accuracy: 0.6991, loss: 0.6849 ||:   1%|          | 45/4546 [00:01<02:33, 29.24it/s]


accuracy: 0.6990, loss: 0.6840 ||:   1%|          | 50/4546 [00:01<02:15, 33.17it/s]


accuracy: 0.6991, loss: 0.6817 ||:   1%|          | 56/4546 [00:01<01:59, 37.44it/s]


accuracy: 0.6992, loss: 0.6791 ||:   1%|▏ 

KeyboardInterrupt: ignored

In [0]:
predictor=Predictor(model,dataset_reader=reader)
tag_logits=predictor.predict_batch_instance(train_dataset[:32])
for i in range(32):
    print(tag_logits[i])
    print(train_dataset[i])

Encountered the loss key in the model's return dictionary which couldn't be split by the batch size. Key will be ignored.


{'tag_logits': [1.6188334226608276, 0.7891334295272827, -1.6063138246536255]}
Instance with fields:
 	 claim: TextField of length 5 with text: 
 		[Lisa, Kudrow, was, in, movies.]
 		and TokenIndexers : {'sentence': 'SingleIdTokenIndexer'} 
 	 evidence: TextField of length 115 with text: 
 		[Away, from, television, ,, Kudrow, has, also, appeared, in, many, films, ,, including, Romy, and,
		Michele, 's, High, School, Reunion, -LRB-, 1997, -RRB-, ,, The, Opposite, of, Sex, -LRB-, 1998,
		-RRB-, ,, Analyze, This, -LRB-, 1999, -RRB-, and, its, sequel, Analyze, That, -LRB-, 2002, -RRB-, ,,
		Dr., Dolittle, 2, -LRB-, 2001, -RRB-, ,, Wonderland, -LRB-, 2003, -RRB-, ,, Happy, Endings, -LRB-,
		2005, -RRB-, ,, P.S., I, Love, You, -LRB-, 2007, -RRB-, ,, Bandslam, -LRB-, 2008, -RRB-, ,, Hotel,
		for, Dogs, -LRB-, 2009, -RRB-, ,, Easy, A, -LRB-, 2010, -RRB-, ,, Neighbors, -LRB-, 2014, -RRB-, ,,
		its, sequel, Neighbors, 2, :, Sorority, Rising, -LRB-, 2016, -RRB-, and, The, Girl, on, the, Train,
	